# Demand Forecasting Tutorial:
## Data Preparation using Python, Pandas, and Psycopg2 for data in a PostgreSQL database

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import psycopg2 as pg
from src.ts_data_preparation import *

In [2]:
# set pandas configuration to show all columns
pd.set_option('display.max_columns', None)

In [3]:
# connect to PostgreSQL database on azure
# connection string information 
host = "tutorial-db.postgres.database.azure.com"
dbname = "postgres"
user = "GTM_admin@tutorial-db"
password = "Enablement1"
sslmode = "require"
port = '5432'

In [4]:
# Construct connection string
conn_string = f"host={host} user={user} dbname={dbname} password={password} sslmode={sslmode}"
conn = pg.connect(conn_string) 
print("Connection established")

cursor = conn.cursor()

Connection established


In [5]:
# get all table names
tag = 'demand_forecasting'
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
tables = [x[0] for x in cursor.fetchall() if tag in str(x)]
print(tables)

['demand_forecasting_items', 'demand_forecasting_transactions_contents', 'demand_forecasting_aggregate_sales', 'demand_forecasting_transaction_status', 'demand_forecasting_market_data']


### We already created the demand_forecasting_aggregate_sales column directly using a SQL query, but ignore that for now. Lets look at other ways to create this using python

# All joins could be performed directly in the PostgreSQL database, but is performed here using Python code and Pandas

## Create a dataframe from each of the PSQL tables

In [6]:
# create a dataframe for each of the tables
df_items = create_df_from_query(table_name= 'demand_forecasting_items', cursor= cursor, index_col= 'index')
df_contents = create_df_from_query(table_name= 'demand_forecasting_transactions_contents', cursor= cursor, index_col= 'index')
df_status = create_df_from_query(table_name= 'demand_forecasting_transaction_status', cursor= cursor, index_col= 'index')
df_markets = create_df_from_query(table_name= 'demand_forecasting_market_data', cursor= cursor, index_col= 'index')

## We can briefly explore the datatsets and then make all joins required to merge this into a single, flat table for modeling

In [7]:
df_items.head()

,package_id,item_name
index,,
0,1700000002,Snickers
1,1700000003,Snickers Crisper
2,1700000004,Hersheys Krackel
3,1700000005,3 Musketeers
4,1700000006,Reeses Peanut Butter cup


In [8]:
df_status.head()

,invoice,date,day,time,transaction,category,store_category,campaign
index,,,,,,,,
0,23137,2019-05-21,Tue,14:35:00,Good,Retail,Type_A,discount_type_1
1,23136,2019-05-21,Tue,14:28:00,Good,Retail,Type_A,discount_type_2
2,23135,2019-05-21,Tue,14:24:00,Good,Retail,Type_A,discount_type_1
3,23134,2019-05-21,Tue,14:21:00,Good,Retail,Type_A,discount_type_3
4,23134,2019-05-21,Tue,14:21:00,Good,Retail,Type_A,discount_type_3


In [9]:
df_contents.head()

,invoice,package_id,qty,unit_price,sale_amount
index,,,,,
0,23137,A000006690,0.28,3.70,1.04
1,23136,A000006815,0.47,2.33,1.10
2,23135,A000006608,1.66,2.07,3.44
3,23134,A000006877,1.66,1.29,2.14
4,23134,A000006799,1.66,1.29,2.14


In [10]:
df_markets.head()

,Month,category_type,metro_area,county_1,county_2,county_3,county_4,county_5,county_6,total_market_sales
index,,,,,,,,,,
0,2015-10-01,Type_A,20546418.0,16398610.0,1743830.0,1881358.0,435580.0,NaN,87040.0,41092836.0
1,2015-10-01,Type_B,25067072.0,19085375.0,NaN,3091341.0,2446592.0,443764.0,NaN,50134144.0
2,2015-03-01,Type_A,18393590.0,14406613.0,1562845.0,1861813.0,479975.0,NaN,82344.0,36787180.0
3,2015-03-01,Type_B,3997657.0,862396.0,NaN,1020552.0,1497796.0,616913.0,NaN,7995314.0
4,2014-05-01,Type_A,18265403.0,14150620.0,1511922.0,2037535.0,467977.0,NaN,97349.0,36530806.0


#### These reported total market sales are created retrospectively, and would not be known at the start of the month. We should change these into the last day of the month, where it is theoretically more relevant.

In [11]:
# change date to last date of month
df_markets['Month'] = (pd.to_datetime(df_markets['Month']) + pd.tseries.offsets.MonthEnd(1)).astype(object)

### Keys for joins

- df_contents <-> df_items using 'package_id' as key
- df_contents <-> df_status using 'invoice' as key
- df_contents <-> df_markets using 'date' and 'store_type' as keys in df_contents and 'Month' and 'category_type' as keys in df_markets

We have additional information that both of these stores are located in county_1, and we may not want all of the columns

In [12]:
df = df_contents.merge(df_items, how= 'left', on= 'package_id')
df = df.merge(df_status, how= 'left', on= 'invoice')
df = df.merge(df_markets[['Month', 'category_type', 'metro_area', 'county_1', 'total_market_sales']], 
              how= 'left', left_on= ['date', 'store_category'], right_on= ['Month', 'category_type']).drop(['Month', 'category_type'], axis= 1)
df.drop_duplicates(inplace= True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121848 entries, 0 to 369285
Data columns (total 16 columns):
invoice               121848 non-null int64
package_id            121848 non-null object
qty                   121848 non-null float64
unit_price            121848 non-null float64
sale_amount           121848 non-null float64
item_name             121848 non-null object
date                  121848 non-null object
day                   121848 non-null object
time                  121848 non-null object
transaction           121848 non-null object
category              121848 non-null object
store_category        121848 non-null object
campaign              121847 non-null object
metro_area            0 non-null float64
county_1              0 non-null float64
total_market_sales    0 non-null float64
dtypes: float64(6), int64(1), object(9)
memory usage: 15.8+ MB


In [14]:
df.head()

,invoice,package_id,qty,unit_price,sale_amount,item_name,date,day,time,transaction,category,store_category,campaign,metro_area,county_1,total_market_sales
0,23137,A000006690,0.28,3.70,1.04,Reeses pieces,2019-05-21,Tue,14:35:00,Good,Retail,Type_A,discount_type_1,NaN,NaN,NaN
1,23136,A000006815,0.47,2.33,1.10,Reeses pieces,2019-05-21,Tue,14:28:00,Good,Retail,Type_A,discount_type_2,NaN,NaN,NaN
2,23135,A000006608,1.66,2.07,3.44,Snickers Crisper,2019-05-21,Tue,14:24:00,Good,Retail,Type_A,discount_type_1,NaN,NaN,NaN
3,23134,A000006877,1.66,1.29,2.14,Mr Good Bar,2019-05-21,Tue,14:21:00,Good,Retail,Type_A,discount_type_3,NaN,NaN,NaN
7,23134,A000006799,1.66,1.29,2.14,Skittles wildberry,2019-05-21,Tue,14:21:00,Good,Retail,Type_A,discount_type_3,NaN,NaN,NaN


In [15]:
# there are two store categories, lets separate the data for each store
df_A = df[df['store_category'] == 'Type_A'].copy()
df_B = df[df['store_category'] == 'Type_B'].copy()

In [16]:
# write data to file for simplicity
df_A.to_csv('data/store_A_sales.csv')
df_B.to_csv('data/store_B_sales.csv')

## The dataset needs to be aggregated for use in forecasting. Aggregation on a date and item_name will be required

In [17]:
# this function will provide basic aggregation functionality and perform some basic calculations on aggregated columns
# for this dataset, this cell will require ~ 1-2 min to run
df_agg = aggregate_df(df= df, 
                      aggregators= ['date', 'item_name'], 
                      string_columns= ['package_id', 'campaign'], 
                      numeric_columns= ['qty', 'unit_price', 'sale_amount'], 
                      ignore_columns= ['day', 'store_category', 'category', 'transaction', 'metro_area', 'county_1', 'total_market_sales'])

Processing 2 string columns:
*** Processing package_id 1/2 ***
*** Processing campaign 2/2 ***

Processing 3 numeric columns:
*** Processing qty 1/3 ***
*** Processing unit_price 2/3 ***
*** Processing sale_amount 3/3 ***

Processing 7 simple columns:
*** Processing day 1/7 ***
*** Processing store_category 2/7 ***
*** Processing category 3/7 ***
*** Processing transaction 4/7 ***
*** Processing metro_area 5/7 ***
*** Processing county_1 6/7 ***
*** Processing total_market_sales 7/7 ***


In [18]:
df_agg.head()

,date,item_name,package_id_min,package_id_max,package_id_unique,campaign_min,campaign_max,campaign_unique,qty_min,qty_mean,qty_max,qty_stdev,qty_unique,qty_sum,unit_price_min,unit_price_mean,unit_price_max,unit_price_stdev,unit_price_unique,unit_price_sum,sale_amount_min,sale_amount_mean,sale_amount_max,sale_amount_stdev,sale_amount_unique,sale_amount_sum,day,store_category,category,transaction,metro_area,county_1,total_market_sales,aggregated_total_count
0,2017-02-03,3 Musketeers,1700000005,1700000005,1,discount_type_1,discount_type_1,1,0.56,0.56,0.56,NaN,1.0,0.56,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,Fri,Type_B,Retail,Canceled,NaN,NaN,NaN,1
1,2017-02-03,Hersheys Krackel,1700000004,1700000004,1,discount_type_1,discount_type_1,1,0.56,0.56,0.56,NaN,1.0,0.56,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,Fri,Type_B,Retail,Canceled,NaN,NaN,NaN,1
2,2017-02-03,Milky Way Midnight,1700000007,1700000007,1,discount_type_1,discount_type_1,1,0.56,0.56,0.56,NaN,1.0,0.56,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,Fri,Type_B,Retail,Canceled,NaN,NaN,NaN,1
3,2017-02-03,Reeses Miniatures,1700000008,1700000008,1,discount_type_1,discount_type_1,1,0.56,0.56,0.56,NaN,1.0,0.56,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,Fri,Type_B,Retail,Canceled,NaN,NaN,NaN,1
4,2017-02-03,Reeses Peanut Butter cup,1700000006,1700000006,1,discount_type_1,discount_type_1,1,0.56,0.56,0.56,NaN,1.0,0.56,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,Fri,Type_B,Retail,Canceled,NaN,NaN,NaN,1


In [19]:
# write data to file for simplicity
df_agg.to_csv('data/aggregate_sales.csv')

## Alternatively, the data can be aggregated directly from the PostgreSQL database using the following SQL query

In [20]:
# this query will get only transactions from 'Type_A' stores, with 'Good' transactions classified as 'Retail'
with open('data/good_psql_query.txt', 'r') as file:
    query = file.read()

In [21]:
# get column names from the final SELECT statement of the query 
columns_string = '''full_sales.date::DATE,
    MAX(full_sales.day) as day,
    full_sales.item_name,
    (ARRAY_AGG(mmp.count_unique_package_id))[1] as package_id_unique,
    (ARRAY_AGG(mmp.min_package_id))[1] as min_package_id,
    (ARRAY_AGG(mmp.max_package_id))[1] as max_package_id,
    (ARRAY_AGG(mmc.count_unique_campaign))[1] as campaign_unique,
    (ARRAY_AGG(mmc.min_campaign))[1] as min_campaign,
    (ARRAY_AGG(mmc.max_campaign))[1] as max_campaign,
    MIN(full_sales.qty) as qty_min,
    AVG(full_sales.qty) as qty_mean,
    MAX(full_sales.qty) as qty_max,
    STDDEV(full_sales.qty) as qty_stdev,
    COUNT(DISTINCT qty) as qty_unique,
    SUM(qty) as qty_sum,
    MIN(full_sales.unit_price) as unit_price_min,
    AVG(full_sales.unit_price) as unit_price_mean,
    MAX(full_sales.unit_price) as unit_price_max,
    STDDEV(full_sales.unit_price) as unit_price_stdev,
    COUNT(DISTINCT full_sales.unit_price) as unit_price_unique,
    SUM(full_sales.unit_price) as unit_price_sum,
    MIN(full_sales.sale_amount) as sale_amount_min,
    AVG(full_sales.sale_amount) as sale_amount_mean,
    MAX(full_sales.sale_amount) as sale_amount_max,
    STDDEV(full_sales.sale_amount) as sale_amount_stdev,
    COUNT(DISTINCT full_sales.sale_amount) as sale_amount_unique,
    SUM(full_sales.sale_amount) as sale_amount_sum,
    AVG(market_sales.metro_area) as metro_area,
    AVG(market_sales.county_1) as county_1,
    AVG(market_sales.total_market_sales) as total_market_sales'''

processed_columns = [x.replace('\n', '').split('as')[-1].split('.')[-1].lstrip() for x in columns_string.split(',')]
processed_columns[0] = 'date'

In [22]:
# execute SQL query and create dataframe with results
# this runs much faster than the aggregate_df function, but requires that you write a SQL query
df_psql = pd.DataFrame(execute_query(query, conn_string), columns= processed_columns)

Connection established


In [23]:
df_psql.head()

,date,day,item_name,package_id_unique,min_package_id,max_package_id,campaign_unique,min_campaign,max_campaign,qty_min,qty_mean,qty_max,qty_stdev,qty_unique,qty_sum,unit_price_min,unit_price_mean,unit_price_max,unit_price_stdev,unit_price_unique,unit_price_sum,sale_amount_min,sale_amount_mean,sale_amount_max,sale_amount_stdev,sale_amount_unique,sale_amount_sum,metro_area,county_1,total_market_sales
0,2018-10-29,Mon,100 Grand,1,9000000759,9000000759,1,discount_type_1,discount_type_1,6.51,7.905,9.30,1.610807,2,31.62,0.00,0.000,0.00,0.000000,1,0.00,0.00,0.000,0.00,0.000000,1,0.00,NaN,NaN,NaN
1,2018-10-29,Mon,3 Musketeers,1,9000000757,9000000757,1,discount_type_1,discount_type_1,7.44,7.440,7.44,0.000000,1,14.88,0.00,0.000,0.00,0.000000,1,0.00,0.00,0.000,0.00,0.000000,1,0.00,NaN,NaN,NaN
2,2018-10-29,Mon,Air Heads,1,9000000758,9000000758,1,discount_type_1,discount_type_1,3.72,4.185,4.65,0.536936,2,16.74,0.01,0.010,0.01,0.000000,1,0.04,0.04,0.045,0.05,0.005774,2,0.18,NaN,NaN,NaN
3,2018-10-29,Mon,Milky Way,1,9000000766,9000000766,1,discount_type_1,discount_type_1,8.37,8.370,8.37,0.000000,1,16.74,0.00,0.000,0.00,0.000000,1,0.00,0.00,0.000,0.00,0.000000,1,0.00,NaN,NaN,NaN
4,2018-10-29,Mon,Nestle Butterfinger,1,9000000764,9000000764,1,discount_type_1,discount_type_1,4.65,7.440,10.23,3.221615,2,29.76,0.00,0.005,0.01,0.005774,2,0.02,0.00,0.025,0.05,0.028868,2,0.10,NaN,NaN,NaN


In [24]:
# write data to file for simplicity
df_psql.to_csv('data/aggregate_sales_from_sql.csv')

# Data is ready for ready for further analysis and modeling

----------------
## Jay wants me to document what I did and why it didn't work for SparkSQL query for engineering to understand and be able to replicate. He said that we seldom do end-to-end workflows in the product and its difficult to understand when/why things don't work from customers. I should use these projects as a mechanism to help improve the product